In [1]:
import tensorflow as tf
import keras
import os 
import cv2

import numpy as np
from sklearn.metrics import accuracy_score
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from sklearn.model_selection import train_test_split
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip,RandomRotation

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, Dense,BatchNormalization,Activation,Input,AveragePooling2D,Flatten,Dropout

In [2]:
def SepaableConv(x,num_filters, strides, alpha=1.0):
    x=DepthwiseConv2D(kernel_size=3,padding='same')(x)
    x=BatchNormalization(momentum=0.9998)(x)
    x=Activation('relu')(x)
    x=Conv2D(np.floor(num_filters * alpha),kernel_size=(1,1),strides=strides,use_bias=False,padding='same')(x)
    x=BatchNormalization(momentum=0.9998)(x)
    x=Activation('relu')(x)
    return x

In [3]:
def Conv(x,num_filters,kernel_size, strides=1, alpha=1.0):
    x=Conv2D(np.floor(num_filters * alpha),kernel_size=(1,1),strides=strides,use_bias=False,padding='same')(x)
    x=BatchNormalization(momentum=0.9998)(x)
    x=Activation('relu')(x)
    
    return x
    


In [4]:
num_classes=2
inputs =Input(shape =(224,224,3))

In [5]:
x= Conv(inputs,num_filters=32,kernel_size=3,strides=2)
x= SepaableConv(x,num_filters=32,strides=1)

x= Conv(x ,num_filters=64,kernel_size=1,)
x= SepaableConv(x,num_filters=64,strides=2)

x= Conv(x ,num_filters=128,kernel_size=1)
x= SepaableConv(x,num_filters=128,strides=1)

x= Conv(x ,num_filters=256,kernel_size=1)
x= SepaableConv(x,num_filters=256,strides=1)


x= Conv(x ,num_filters=256,kernel_size=1)
x= SepaableConv(x,num_filters=256,strides=2)

x= Conv(x ,num_filters=512,kernel_size=1)

for i in range(5):  
    x= SepaableConv(x,num_filters=512,strides=1)
    x= Conv(x ,num_filters=512,kernel_size=1)
    
x= SepaableConv(x,num_filters=512,strides=1)

x= Conv(x ,num_filters=1024,kernel_size=1)

x=AveragePooling2D(pool_size=(7,7))(x)
x=Flatten()(x)

outputs = Dense(num_classes,activation='softmax')(x)
outputs = Dropout(0.2)(outputs)
outputs = Activation('softmax')(outputs)

In [6]:
model=Model(inputs, outputs)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=Adam(0.001),
             metrics=['acc'])

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      96        
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 112, 112, 32)     320       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 32)     128   

In [8]:
# datasets_directory='hum_hors/'
# categories=['hor','hum_hors']
# img_size=224

datasets_directory='datasets/'
categories=['cats','dogs']
img_size=224

In [9]:
image_augmentation=Sequential([RandomFlip('horizontal_and_vertical'),
                             RandomRotation(0.2)])

In [10]:
trainingDatasets=[]
def create_trainingDatasets():
    for i in categories:
        path = os.path.join(datasets_directory,i)
        class_number=categories.index(i)
        
        for img in os.listdir(path):
            imageArray= cv2.imread(os.path.join(path,img))
            img_size=224
            modifiedArray=cv2.resize(imageArray,(img_size,img_size))
            modifiedArray=tf.expand_dims(modifiedArray,0)
            
        for i in range(500):
            imgAugment = image_augmentation(modifiedArray)
            trainingDatasets.append([modifiedArray,class_number])
            
create_trainingDatasets()

In [11]:
print(len(trainingDatasets)) 

1000


In [12]:
XDatasets = []
yDatasets = []

for feature, labels in trainingDatasets:
    XDatasets.append(feature)
    yDatasets.append(labels)

In [13]:
# XDatasets

In [14]:
XDatasets =np.array(XDatasets).reshape(-1,img_size,img_size,3)
yDatasets = np.array(yDatasets).reshape((-1,1))


In [15]:
XDatasets = XDatasets.astype('float32')
XDatasets/=255

In [16]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(XDatasets,yDatasets,test_size=0.2,random_state=4)

In [17]:
model.fit(xtrain,ytrain,epochs=3,validation_data=(xtest,ytest),shuffle=True)

Epoch 1/3
25/25 [==============================] - 335s 13s/step - loss: 0.8286 - acc: 0.5175 - val_loss: 0.7033 - val_acc: 0.5000
Epoch 2/3
25/25 [==============================] - 330s 13s/step - loss: 0.8277 - acc: 0.5138 - val_loss: 0.6993 - val_acc: 0.5000
Epoch 3/3
25/25 [==============================] - 295s 12s/step - loss: 0.8484 - acc: 0.4875 - val_loss: 0.6952 - val_acc: 0.5000
